In [1]:
from astropy.io import fits
import numpy as np
import pylab as plt
import glob as glob
from scipy.stats import mode
from matplotlib.colors import LogNorm
from transform import get_wcs_solution
from scipy.interpolate import interp1d
import matplotlib.gridspec as gridspec
from scipy.signal import find_peaks
from scipy.signal import medfilt
from scipy.optimize import curve_fit
from scipy.stats import norm
from astropy import units as u
from astropy.modeling import models
from astropy.modeling import fitting



In [7]:
#reading in the dat file
ID, redshift = np.genfromtxt('targets.dat', usecols=(0,1), unpack = True, skip_header=2, dtype = 'str')
z = redshift.astype(float)

line_info = np.genfromtxt('linelist.dat', unpack =True, dtype='str', usecols=(0,1), skip_header = 1)

line_wavelength = line_info[0].astype(float)
line_name = line_info[1]

In [8]:
ID

array(['J030903.87+003846.9', 'J231903.23+010853.5',
       'J073149.48+404513.2', 'J082540.44+184617.2',
       'J021306.61+005612.4', 'J114827.33+254611.7',
       'J232122.50+003455.0', 'J120149.90+280610.6',
       'J014707.04+135629.1'], dtype='<U19')

In [9]:
z

array([0.03 , 0.03 , 0.034, 0.038, 0.04 , 0.045, 0.053, 0.056, 0.057])

In [10]:
line_wavelength

array([ 6562.8,  4861.3,  4340.5,  4101.7,  3970.1,  3889.1,  3835.4,
        3797.9, 10049.4,  9546. ,  9229. ,  9014.9,  4471. ,  3820. ,
        3889. ,  4026. ,  5876. ,  7065. , 10830. ,  6678. ,  4922. ,
        5016. ,  3965. ,  4686. , 10123. ,  6560. ,  5412. ,  4859. ,
        4541. ,  4339. ,  9344. ,  8236. ,  7178. ,  5577. ,  6300. ,
        6363. ,  3725. ,  3727. ,  3869. ,  7320. ,  7330. ,  4363. ,
        4959. ,  5007. ,  5755. ,  6548. ,  6583. ,  6312. ,  6716. ,
        6730. ,  3868. ,  3970. ,  7135. ,  7751. ,  4711. ,  4740. ])

In [11]:
files = [x for x in glob.glob('*.fits') if 'SDSS' not in x]

In [12]:
files

['median_J014707+135629_cem.fits',
 'median_J021306+005612_mods1b_ce.fits',
 'median_J021306+005612_mods1r_ce.fits',
 'median_J030903+003846_cem.fits',
 'median_J073149+404513_mods1b_ce.fits',
 'median_J073149+404513_mods1r_ce.fits',
 'median_J082540+184617_mods1b_ce.fits',
 'median_J082540+184617_mods1r_ce.fits',
 'median_J231903+010853_cem.fits']

In [19]:
file_num = np.unique([x.split('_')[1] for x in files])

In [20]:
file_num

array(['J014707+135629', 'J021306+005612', 'J030903+003846',
       'J073149+404513', 'J082540+184617', 'J231903+010853'], dtype='<U14')

In [15]:
ID1 = [x.split('.')[0] for x in ID]

In [16]:
ID1

['J030903',
 'J231903',
 'J073149',
 'J082540',
 'J021306',
 'J114827',
 'J232122',
 'J120149',
 'J014707']

In [23]:
test = np.zeros(len(ID1), dtype = bool)

for j, val in enumerate(ID1):
    for i in file_num:
        if val in i:
            test[j] = True  
            

In [24]:
test

array([ True,  True,  True,  True,  True, False, False, False,  True])

In [33]:
ID_1 = np.array(ID1)[test]

In [34]:
z_redshift = np.array(z)[test]

In [36]:
for i in files:
    #print(type(i))
    sp1 = i.split('_')[1]
    sp2 = sp1.split('+')[0]
    
    match = ID_1 == sp2
    print(match)

[False False False False False  True]
[False False False False  True False]
[False False False False  True False]
[ True False False False False False]
[False False  True False False False]
[False False  True False False False]
[False False False  True False False]
[False False False  True False False]
[False  True False False False False]


In [8]:
cat spectrum_lines.txt

LineCent LineName
------- ----------
3187.74 HeI_3188
3203.10 HeII_3203
3426.85 [NeV]3427
3703.30 H16
3711.97 H15
3721.94 H14
3725.00 [OII]3725
3727.00 [OII]3727
3750.15 H12_3750
3770.63 H11_3771
3797.90 H10_3798
3819.64 HeI_3819
3835.4  Hn
3868.76 [NeIII]3869
3889.1  HeI+H8
3968.00 [NeIII]+H7
4026.19 HeI_4026
4068.60 [SII]4068
4076.35 [SII]4076
4101.74 Hd_4101
4120.84 HeI_4121
4143.76 HeI_4144
4227.20 [FeV]4227
4340.47 Hg+HeII_10-4
4363.21 [OIII]4363
4387.93 HeI_4388
4471.48 HeI_4471
4541.00 HeII_4541
4658.10 [FeIII]4658
4685.94 HeII_4686
4712.00 [ArIV]+HeI
4740.20 [ArIV]4740
4859.00 HeII_4859
4861.33 Hbeta_4861
4921.93 HeI_4922
4958.92 [OIII]4959
4988.00 [FeIII]4988
5006.80 [OIII]5007
5015.68 HeI_5015
5411.52 HeII_5411
5517.71 [ClIII]5518
5537.88 [ClIII]5538
5577.00 [OI]5577
5754.64 [NII]5754
5875.60 HeI_5876
6300.30 [OI]6300
6312.10 [SIII]6312
6363.80 [OI]6363
6548.10 [NII]6548
6562.80 Halpha_6562
6583.40 [NII]6583
6678.10 HeI_6678
6716.40 [SII]6716
6730.80 [SII]6730
7065.30 HeI_706

# Working on an improved line analysis tool

In [37]:
line, line_name = np.genfromtxt('spectrum_lines.txt', unpack = True, skip_header = 2, usecols = [0, 1], dtype = str)

In [38]:
line_wavelength = line.astype(float)

In [39]:
line_wavelength

array([3187.74, 3203.1 , 3426.85, 3703.3 , 3711.97, 3721.94, 3725.  ,
       3727.  , 3750.15, 3770.63, 3797.9 , 3819.64, 3835.4 , 3868.76,
       3889.1 , 3968.  , 4026.19, 4068.6 , 4076.35, 4101.74, 4120.84,
       4143.76, 4227.2 , 4340.47, 4363.21, 4387.93, 4471.48, 4541.  ,
       4658.1 , 4685.94, 4712.  , 4740.2 , 4859.  , 4861.33, 4921.93,
       4958.92, 4988.  , 5006.8 , 5015.68, 5411.52, 5517.71, 5537.88,
       5577.  , 5754.64, 5875.6 , 6300.3 , 6312.1 , 6363.8 , 6548.1 ,
       6562.8 , 6583.4 , 6678.1 , 6716.4 , 6730.8 , 7065.3 , 7135.8 ,
       7178.  , 7281.21, 7319.9 , 7330.2 , 7751.12, 8236.  , 9014.9 ,
       9069.  , 9229.  , 9344.  , 9530.6 , 9546.  ])

In [32]:
line_name

array(['HeI_3188', 'HeII_3203', '[NeV]3427', 'H16', 'H15', 'H14',
       '[OII]3725', '[OII]3727', 'H12_3750', 'H11_3771', 'H10_3798',
       'HeI_3819', 'Hn', '[NeIII]3869', 'HeI+H8', '[NeIII]+H7',
       'HeI_4026', '[SII]4068', '[SII]4076', 'Hd_4101', 'HeI_4121',
       'HeI_4144', '[FeV]4227', 'Hg+HeII_10-4', '[OIII]4363', 'HeI_4388',
       'HeI_4471', 'HeII_4541', '[FeIII]4658', 'HeII_4686', '[ArIV]+HeI',
       '[ArIV]4740', 'HeII_4859', 'Hbeta_4861', 'HeI_4922', '[OIII]4959',
       '[FeIII]4988', '[OIII]5007', 'HeI_5015', 'HeII_5411',
       '[ClIII]5518', '[ClIII]5538', '[OI]5577', '[NII]5754', 'HeI_5876',
       '[OI]6300', '[SIII]6312', '[OI]6363', '[NII]6548', 'Halpha_6562',
       '[NII]6583', 'HeI_6678', '[SII]6716', '[SII]6730', 'HeI_7065',
       '[ArIII]7136', 'HeII_7178', 'HeI_7281', '[OII]7320', '[OII]7330',
       '[ArIII]7751', 'HeII_8236', 'P10', '[SIII]9069', 'P9', 'HeII_9344',
       '[SIII]9530', 'P8'], dtype='<U12')

In [33]:
def fitting_lines(spectrum, wavelength, sig = 4):
    '''
    This function fits the emission lines from a spectrum object. The way that this is used for is if you have a sub_spectrum 
    of where you think the emission line is at +/- a window.
    
    Ie. if you have a line at 5009 angstroms and a window of 10 angstroms then the spectrum passed in must be a 
    subspectrum between 4999 and 5019.
    
    Essentially just your emission line

    Parameters
    ------------------------
    spectrum: This is a sub_spectrum that holds your emission line

    Returns
    -----------------------
    param: this is the parameters for the gaussian function param = [Amplitude, Mean, Sigma]
    '''

    #defining the gaussian function I would like to fit to the data
    def f(x, A, mu, sig):
        return A * np.exp(-(x-mu)**2/(2*sig**2))

    #code that tries to fit the data with a gaussian
    try:
        #gets the mean value from the spectrum passed in
        mu = np.mean(wavelength)
        #performs the fitting
        param, covar = curve_fit(f, wavelength, spectrum,
                                 p0 = [np.amax(spectrum), mu, sig])
        return param

    #If it could not fit the data and we get a Runtime Error we return an array of -999.
    #We can use this as a filter because we know that wavelengths won't be negative, so we get a param[1] < 0
    except RuntimeError:
        return np.array([-999,-999,-999])
    except OptimizeWarning:
        return np.array([-1, -1, -1])

In [34]:
def Monte_Carlo(wavelength, flux, noise, line_center, continuum_func, filt):

    '''

    This function will run a monte carlo simulation of a run of 1000 samples and wil calculate the
    fux and equivalent width of emission lines of interest

    Parameters
    ------------------------
    wavelength: This is the wavelength of the noise filtered spectrum
    flux: The flux of the spectrum with a noise filtered applied to get rid of noisy outer regions
    distribution: This is the extraction standard deviation and should be in the form of a 1D array
    line_center: The line center that we will be calculating the line flux and EW
    continuum_func: The function used to get the continuum as this will be used to get the EW

    Returns
    ------------------------
    emission_line_center: the emission line center in wavelength
    emission_line_center_err: the error in the line center calculation
    manual_ew: The EW of the line
    manual_ew_err: the respective error
    manual_flux: The flux of the line
    manual_flux_err: the relative error on the flux

    '''
    
    filt_flux = flux[filt]
    filt_wave = wavelength[filt]
    
    i = line_center

    dist_flux = []
    dist_ew = []
    dist_center = []
    SN = -999

    counter = 0
    bad_fit = 0

    #print(counter)
    sn_counter = 0
    while counter <= 1000:
        
        #print(counter)
        new_flux = np.random.normal(filt_flux, scale = noise[filt])


        #making a spectrum object
        #spectrum = Spectrum1D(spectral_axis=wavelength*u.angstrom,
                                  #flux =new_flux*u.erg/u.s/u.cm/u.cm/u.angstrom)

        window = 6

        #looking at the sub_region around where the line center is located at and +/- 6 Angstroms
        if (i + window) > filt_wave[-1]:
            
            return np.array([-999,-999,-999, -999, -999, -999])
        if (i - window) < filt_wave[0]:
            
            return np.array([-999,-999,-999, -999, -999, -999])
        
        #getting the indices where the wavelength array satisfies the condition
        index = np.where(((i - window) >= filt_wave) & ((i + window) <= filt_wave))
        
        line_flux_region = new_flux[index]
        
        line_wavelength_region = filt_wave[index]


        '''
        if counter == 0:
            spectrum1 = Spectrum1D(spectral_axis=wavelength*u.angstrom,
                                  flux =flux*u.erg/u.s/u.cm/u.cm/u.angstrom, uncertainty=uncertainty)
            window = 8*u.angstrom

            #looking at the sub_region around where the line center is located at and +/- 10 Angstroms
            sub_region1 = SpectralRegion(i - window, i + window)
            sub_spectrum1 = extract_region(spectrum1, sub_region1)
            SN = round(snr(sub_spectrum1).value, 2)
        '''
        
        #this calls a function which fits the sub region with a gaussian
        par = fitting_lines(sub_spectrum)

        #############
        #Note that if for some reason a gaussian cannot be fit it will return values of [-999, -999, -999] and
        #we do not want those so we can omit these
        #essentially we will not be fitting the line and getting a flux value or EW
        #############

        #print(par[1])
        #checks to make sure fit worked if not it skips it

        if par[1] < 0:
            bad_fit += 1
            #counter += 1
            if bad_fit > 700:
                return np.array([-999,-999,-999, -999, -999, -999])
            continue
        
        if sn_counter == 0:
            
            wave = wavelength[filt]
            noise_flux = flux[filt]
            
            index = abs(i.value - wave).argmin()
            
            rms_noise = noise_flux[index-7:index+7]
            
            std_noise = np.std(rms_noise)
            
            SN = round(par[0]/std_noise, 4)
            sn_counter += 1
            
        #Sn = snr(sub_spectrum)
        #getting the flux of the line using scipy curve fit parameters
        flux_line = np.sqrt(2*np.pi)*abs(par[0])*abs(par[-1])

        #getting the equivalent width from flux calculation above
        dist_ew.append(flux_line/continuum_func(par[1]*u.angstrom).value)

        #getting the center of the emission peak from curve_fit and appending it
        #emission_line_center.append(par[1])

        #line_center_index.append(abs(spectrum.spectral_axis.value - par[1]).argmin())
        #if counter == 0 or counter == 500:
            #x = np.linspace(sub_spectrum.spectral_axis.value[0], sub_spectrum.spectral_axis.value[-1], 1000)
            #plt.plot(x, f(x, *par))
            #plt.plot(sub_spectrum.spectral_axis, sub_spectrum.flux)
        #appending the manual flux calculations
        dist_flux.append(flux_line)

        dist_center.append(par[1])

        #SN_dist.append(Sn)

        counter += 1

        #if counter == 1000:
         #   break


    #print(counter)
    
    if len(dist_center) == 0:
        return -999 * np.ones(6)

    low = np.percentile(dist_flux, q=1, interpolation='midpoint')
    high = np.percentile(dist_flux, q=99, interpolation='midpoint')

    good_data = np.where((dist_flux>low) & (dist_flux < high))

    hist_center, bin_edges_center = np.histogram(np.array(dist_center)[good_data], bins='fd')

    #plt.plot(bin_edges_center[:-1], dist_center, '.')

    hist_flux, bin_edges_flux = np.histogram(np.array(dist_flux)[good_data], bins = 'fd')

    #plt.hist(dist_flux, bins = len(bin_edges_flux))
    #plt.show()

    hist_ew, bin_edges_ew = np.histogram(np.array(dist_ew)[good_data], bins = 'fd')

    #hist_SN, bin_edges_SN = np.histogram(SN_dist, bins='auto')

    #plt.hist(dist_ew, bins = len(bin_edges_ew))
    #plt.show()

    #print(len(bin_edges_center))
    #print(len(hist_center))
    bin_center = .5 * (bin_edges_center[1:] + bin_edges_center[:-1])
    #print(len(bin_center))

    #plt.plot(bin_center, hist_center, '.')

    bin_center_flux = .5 * (bin_edges_flux[1:] + bin_edges_flux[:-1])

    bin_center_ew = .5 * (bin_edges_ew[1:] + bin_edges_ew[:-1])
    #bin_center_SN = .5 * (bin_edges_SN[1:] + bin_edges_SN[:-1])

    def f(x, A, mu, sig):
        return A * np.exp(-(x-mu)**2/(2*sig**2))


    emission_line_center = -999
    par_center = -999 * np.ones(3)
    par_flux = -999 * np.ones(3)
    par_ew = -999 * np.ones(3)
    #par_SN = -999 * np.ones(3)

    try:
        #print('Emission Lines')
        par_center, cov = curve_fit(f, bin_center, hist_center, p0 = [np.amax(hist_center), bin_center[hist_center.argmax()], np.std(bin_center)])

        emission_line_center = round(par_center[1], 4)

    except RuntimeError:
        #print('emission Line err')
        emission_line_center = round(line_center.value, 4)
    #fig, (ax1, ax2) = plt.subplots(2, 1)
    #ax1.bar(bin_center_flux, hist_flux, align='center', width=bin_center_flux[1]-bin_center_flux[0])
    #ax2.bar(bin_center_ew, hist_ew, align='center', width=bin_center_ew[1]-bin_center_ew[0])
    #plt.show()
    
    try:
        #print('calculation')
        par_flux, cov = curve_fit(f, bin_center_flux, hist_flux, p0 = [np.amax(hist_flux), bin_center_flux[hist_flux.argmax()], np.std(bin_center_flux)])
        #print('after flux')

    except RuntimeError:
        #print('calculation error flux')
        return -999*np.ones(6)
    try:
        par_ew, cov = curve_fit(f, bin_center_ew, hist_ew, p0 = [np.amax(hist_ew), bin_center_ew[hist_ew.argmax()], np.std(bin_center_ew)])
        #print('after ew')
        #par_SN, cov = curve_fit(f, bin_center_SN, hist_SN, p0 = [np.amax(hist_SN), bin_center_SN[hist_SN.argmax()], np.std(bin_center_SN)])

    except RuntimeError:
        #print('calculation error EW')
        return -999*np.ones(6)
    '''
    try:
        par_center, cov = curve_fit(f, bin_center, hist_center, p0 = [np.amax(hist_center), bin_center[hist_center.argmax()], np.std(bin_center)])
    '''

    #par_flux, cov = curve_fit(f, bin_center_flux, hist_flux, p0 = [np.amax(hist_flux), bin_center_flux[hist_flux.argmax()], np.std(bin_center_flux)])
    #par_ew, cov = curve_fit(f, bin_center_ew, hist_ew, p0 = [np.amax(hist_ew), bin_center_ew[hist_ew.argmax()], np.std(bin_center_ew)])
    #par_SN, cov = curve_fit(f, bin_center_SN, hist_SN, p0 = [np.amax(hist_SN), bin_center_SN[hist_SN.argmax()], np.std(bin_center_SN)])
    
    #x = np.linspace(bin_center_flux[0], bin_center_flux[-1], 1000)
    #fit = f(x, *par_flux)
    

    #emission_line_center = round(par_center[1], 2)
    #emission_line_center_err = round(par_center[-1], 4)
    manual_ew = round(par_ew[1], 4)
    manual_ew_err = round(par_ew[-1], 4)
    manual_flux = round(par_flux[1], 4)
    manual_flux_err = round(par_flux[-1], 4)
    #SN = round(par_SN[1], 1)
    #SN_err = round(par_SN[-1], 4)
    
    '''
    if p <= 3:
        x = np.linspace(bin_center_flux[0], bin_center_flux[-1], 1000)
        fit = f(x, *par_flux)
        #np.savez(str(round(i.value))+'_flux.npz', bin_center=bin_center_flux, flux_dist=hist_flux, fit_flux=fit, xfit = x)
        x1 = np.linspace(bin_center_ew[0], bin_center_ew[-1], 1000)
        fit_ew = f(x1, *par_ew)
        #np.savez(str(round(i.value))+'_ew.npz', bin_center_ew=bin_center_ew, ew_dist=hist_ew, fit_ew=fit_ew, xfit=x1)

    
    emission_line_center = np.nanmean(dist_center)
    emission_line_center_err = np.nanstd(dist_center)

    manual_ew = np.nanmean(dist_ew)
    manual_ew_err = np.nanstd(dist_ew)

    manual_flux = np.nanmean(dist_flux)
    manual_flux_err = np.nanstd(dist_flux)
    '''
    
    #print('Everything Gucci')
    
    return emission_line_center, manual_ew, manual_ew_err, manual_flux, manual_flux_err, SN

In [35]:
def analysis(wavelength, flux, noise, line_center, continuum_func, filt):
    
    line_center_wave = []
    
    calculated_flux = []
    flux_err = []
    
    calculated_EW = []
    EW_err = []
    
    
    for line in line_center:
        
        emission_center_wave, manual_ew, manual_ew_err, manual_flux, manual_flux_err, SN = Monte_Carlo(wavelength, flux, noise, line, continuum_func, filt)
        
        line_center_wave.append(emission_center_wave)
        
        calculated_flux.append(manual_flux)
        flux_err.append(manual_flux_err)
        
        calculated_EW.append(manual_ew)
        EW_err.append(manual_ew_err)
        
        
        

['SpecJ082540+184617_mods1b.npz',
 'SpecJ030903+003846_cem.npz',
 'SpecJ082540+184617_mods1r.npz',
 'SpecJ014707+135629_cem.npz',
 'Subtracted_Spectrum.npz',
 'SpecJ231903+010853_cem.npz',
 'SpecJ021306+005612_mods1b.npz',
 'Subtracted_Spectrum_Red.npz',
 'SpecJ021306+005612_mods1r.npz',
 'SpecJ073149+404513_mods1b.npz',
 'SpecJ073149+404513_mods1r.npz']

In [88]:
def getting_filenames():
    npz_files = [x for x in glob.glob('*.npz') if 'Spec' in x and 'error' not in x and 'continuum' not in x]
    new_npz_list = npz_files[:4] + npz_files[5:7] + npz_files[8:]
    return new_npz_list

In [89]:
files = getting_filenames()

In [90]:
files

['SpecJ082540+184617_mods1b.npz',
 'SpecJ030903+003846_cem.npz',
 'SpecJ082540+184617_mods1r.npz',
 'SpecJ014707+135629_cem.npz',
 'SpecJ231903+010853_cem.npz',
 'SpecJ021306+005612_mods1b.npz',
 'SpecJ021306+005612_mods1r.npz',
 'SpecJ073149+404513_mods1b.npz',
 'SpecJ073149+404513_mods1r.npz']

In [91]:
u = np.load(files[2][:-4] + '_continuum'+ files[2][-4:], allow_pickle=True)

In [92]:
u['continuum']

RecursionError: maximum recursion depth exceeded

In [ ]:
#Getting the necessary data from the files

In [ ]:
for file in files:
    
    loaded_data = np.load(file)
    
    flux = loaded_data['flux']
    wavelength = loaded_data['wavelength']
    redshift = loaded_data['z']
    
    error_file = file[:-4] + '_error'+ file[-4:]
    
    error_loaded_data = np.load(error_file)
    error_data = error_loaded_data['err']
    
    continuum_file = file[:-4] + '_continuum' + file[-4:]
    
    loaded_continuum_data = np.load(continuum_file, allow_pickle=True)
    continuum_data = loaded_continuum_data['continuum'].item()
    